# TranSysTor IDE Complet 🔷

**Framework TSCP v0.2.0 - Version Tout-en-Un**

Toutes les fonctionnalités dans un seul notebook :
- 🎲 Cubes imbriqués CM1/CM2/CM3
- 📐 Plan CM0 (5×5)
- 🌐 Multilingue FR/EN
- 📊 Export OWL/RDFS/SHACL
- 💬 Chatbot IA
- 🧮 Validation mathématique

## 1. Configuration et imports

In [1]:
# Imports standard
import json
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from datetime import datetime

# Chemins
MODEL_DIR = Path('../models/tscp')
EXPORT_DIR = Path('../exports')

# Créer les dossiers
MODEL_DIR.mkdir(parents=True, exist_ok=True)
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Imports réussis")
print(f"📂 Modèles: {MODEL_DIR.absolute()}")
print(f"📂 Exports: {EXPORT_DIR.absolute()}")

✅ Imports réussis
📂 Modèles: E:\_00_Michel\_00_Lab\_00_GitHub\transystor\notebooks\..\models\tscp
📂 Exports: E:\_00_Michel\_00_Lab\_00_GitHub\transystor\notebooks\..\exports


## 2. Classes et configuration

In [2]:
# État global
class IDEState:
    def __init__(self):
        self.language = 'fr'
        self.visible_layers = {'CM0': False, 'CM1': True, 'CM2': True, 'CM3': False}
        self.exclusive_layer = 'CM2'
        self.show_grid = True
        self.show_axes = True

state = IDEState()

# Traductions
TRANSLATIONS = {
    'fr': {'title': 'TranSysTor IDE - Cubes Imbriqués'},
    'en': {'title': 'TranSysTor IDE - Nested Cubes'}
}

def t(key):
    return TRANSLATIONS[state.language].get(key, key)

# Configuration des cubes
CUBE_CONFIGS = {
    'CM0': {'type': 'plane', 'z': -0.5, 'color': '#4b5563'},
    'CM1': {'size': 3, 'center': [2, 2, 2], 'color': '#3b82f6'},
    'CM2': {'size': 4, 'center': [2, 2, 2], 'color': '#10b981'},
    'CM3': {'size': 5, 'center': [2, 2, 2], 'color': '#8b5cf6'}
}

print("✅ Configuration chargée")

✅ Configuration chargée


## 3. Données - Principes

In [3]:
principles_data = [
    # CM0 - Plan
    {'name': 'Processus', 'layer': 'CM0', 'position': [1, 1, -0.5], 'color': '#ef4444',
     'description': 'Transformation dans le temps'},
    {'name': 'Structure', 'layer': 'CM0', 'position': [2, 2, -0.5], 'color': '#f59e0b',
     'description': 'Organisation spatiale'},
    {'name': 'Échange', 'layer': 'CM0', 'position': [3, 3, -0.5], 'color': '#10b981',
     'description': 'Transfert d\'information'},
    
    # CM1 - Cube 3×3×3
    {'name': 'Observateur', 'layer': 'CM1', 'position': [1.5, 1.5, 1.5], 'color': '#3b82f6',
     'description': 'Point de départ anthropocentrique'},
    {'name': 'Interface', 'layer': 'CM1', 'position': [2, 1.5, 2], 'color': '#10b981',
     'description': 'Médiation entre systèmes'},
    {'name': 'Réseau', 'layer': 'CM1', 'position': [2.5, 2, 2.5], 'color': '#ef4444',
     'description': 'Structure organisationnelle'},
    
    # CM2 - Cube 4×4×4
    {'name': 'Protocole', 'layer': 'CM2', 'position': [2, 1, 1], 'color': '#a855f7',
     'description': 'Instance d\'Interface avec règles'},
    {'name': 'Bus', 'layer': 'CM2', 'position': [1, 3, 3], 'color': '#f97316',
     'description': 'Bus = Processus ⊗ Distribution', 'combination': True},
]

print(f"📦 {len(principles_data)} principes chargés")

📦 8 principes chargés


## 4. Fonction de visualisation

In [4]:
def create_viz(principles, show_layers, exclusive=None, grid=True, axes=True):
    fig = go.Figure()
    
    # Plan CM0
    if show_layers.get('CM0') and (not exclusive or exclusive == 'CM0'):
        z = -0.5
        for i in range(6):
            fig.add_trace(go.Scatter3d(
                x=[0, 5], y=[i, i], z=[z, z], mode='lines',
                line=dict(color='gray', width=2), opacity=0.3,
                showlegend=False, hoverinfo='skip'
            ))
            fig.add_trace(go.Scatter3d(
                x=[i, i], y=[0, 5], z=[z, z], mode='lines',
                line=dict(color='gray', width=2), opacity=0.3,
                showlegend=False, hoverinfo='skip'
            ))
    
    # Fonction cube
    def draw_cube(size, center, color, layer):
        if not show_layers.get(layer) or (exclusive and exclusive != layer):
            return
        half = size / 2
        cx, cy, cz = center
        v = [[cx-half, cy-half, cz-half], [cx+half, cy-half, cz-half],
             [cx+half, cy+half, cz-half], [cx-half, cy+half, cz-half],
             [cx-half, cy-half, cz+half], [cx+half, cy-half, cz+half],
             [cx+half, cy+half, cz+half], [cx-half, cy+half, cz+half]]
        edges = [[0,1],[1,2],[2,3],[3,0],[4,5],[5,6],[6,7],[7,4],[0,4],[1,5],[2,6],[3,7]]
        lines = []
        for e in edges:
            lines.extend([v[e[0]], v[e[1]], [None, None, None]])
        x, y, z = zip(*lines)
        fig.add_trace(go.Scatter3d(
            x=x, y=y, z=z, mode='lines',
            line=dict(color=color, width=3), opacity=0.6,
            name=f'{layer} ({size}×{size}×{size})', hoverinfo='name'
        ))
    
    # Dessiner cubes
    for layer in ['CM1', 'CM2', 'CM3']:
        cfg = CUBE_CONFIGS[layer]
        draw_cube(cfg['size'], cfg['center'], cfg['color'], layer)
    
    # Principes
    for p in principles:
        if exclusive and p['layer'] != exclusive:
            continue
        if not exclusive and not show_layers.get(p['layer']):
            continue
        pos = p['position']
        fig.add_trace(go.Scatter3d(
            x=[pos[0]], y=[pos[1]], z=[pos[2]],
            mode='markers+text',
            marker=dict(size=10, color=p['color'], line=dict(color='white', width=2)),
            text=[p['name']], textposition='top center',
            name=p['name'], hovertext=p['description']
        ))
    
    # Axes
    if axes:
        for axis, color, end in [('I', 'red', [5.5,0,0]), ('J', 'green', [0,5.5,0]), ('K', 'blue', [0,0,5.5])]:
            fig.add_trace(go.Scatter3d(
                x=[0, end[0]], y=[0, end[1]], z=[0, end[2]],
                mode='lines+text', line=dict(color=color, width=4),
                text=['', axis], textfont=dict(size=14, color=color),
                showlegend=False, hoverinfo='skip'
            ))
    
    fig.update_layout(
        title=t('title'),
        scene=dict(
            xaxis=dict(range=[-0.5, 5.5], showgrid=False),
            yaxis=dict(range=[-0.5, 5.5], showgrid=False),
            zaxis=dict(range=[-1, 5.5], showgrid=False),
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.3)),
            aspectmode='cube'
        ),
        height=700,
        paper_bgcolor='#1f2937',
        font=dict(color='white')
    )
    return fig

print("✅ Fonction de visualisation créée")

✅ Fonction de visualisation créée


## 5. Interface Interactive

In [5]:
# Widgets
layer_checks = {
    'CM0': widgets.Checkbox(value=False, description='CM0 (Plan 5×5)'),
    'CM1': widgets.Checkbox(value=True, description='CM1 (Cube 3×3×3)'),
    'CM2': widgets.Checkbox(value=True, description='CM2 (Cube 4×4×4)'),
    'CM3': widgets.Checkbox(value=False, description='CM3 (Cube 5×5×5)')
}

exclusive = widgets.Dropdown(
    options=[('Superposé', None), ('CM0', 'CM0'), ('CM1', 'CM1'), ('CM2', 'CM2'), ('CM3', 'CM3')],
    value='CM2', description='Mode:'
)

lang = widgets.ToggleButtons(
    options=[('🇫🇷 FR', 'fr'), ('🇬🇧 EN', 'en')],
    value='fr'
)

grid_check = widgets.Checkbox(value=True, description='Grille')
axes_check = widgets.Checkbox(value=True, description='Axes')

update_btn = widgets.Button(
    description='🔄 Actualiser',
    button_style='primary'
)

viz_out = widgets.Output()

def update_viz(b=None):
    state.language = lang.value
    state.exclusive_layer = exclusive.value
    for k, v in layer_checks.items():
        state.visible_layers[k] = v.value
    
    with viz_out:
        clear_output(wait=True)
        fig = create_viz(
            principles_data,
            state.visible_layers,
            state.exclusive_layer,
            grid_check.value,
            axes_check.value
        )
        fig.show()

update_btn.on_click(update_viz)

display(widgets.VBox([
    widgets.HTML("<h2>⚙️ Contrôles</h2>"),
    lang, exclusive,
    widgets.HTML("<h3>Couches:</h3>"),
    *layer_checks.values(),
    widgets.HTML("<h3>Options:</h3>"),
    grid_check, axes_check,
    update_btn,
    viz_out
]))

update_viz()

## 6. Export OWL

In [6]:
def export_owl(principles):
    owl = """@prefix : <http://transystor.org/ontology/tscp#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

:TSCP rdf:type owl:Ontology ;
    rdfs:label "TSCP Ontology" ;
    owl:versionInfo "0.2.0" .
"""
    for p in principles:
        name = p['name'].replace(' ', '_')
        owl += f"\n:{name} rdf:type :CM{p['layer'][-1]}_Class ;\n"
        owl += f'    rdfs:label "{p["name"]}" ;\n'
        owl += f'    :hasPosition "{p["position"]}" .\n'
    return owl

export_btn = widgets.Button(description='📊 Exporter OWL', button_style='success')
export_out = widgets.Output()

def do_export(b):
    with export_out:
        clear_output()
        content = export_owl(principles_data)
        filename = f"tscp_{datetime.now().strftime('%Y%m%d_%H%M%S')}.ttl"
        filepath = EXPORT_DIR / filename
        with open(filepath, 'w') as f:
            f.write(content)
        print(f"✅ Exporté: {filepath}")
        print(f"\n{content[:500]}...")

export_btn.on_click(do_export)
display(widgets.VBox([export_btn, export_out]))

## 7. Validation mathématique

In [7]:
def compute_ortho(principles):
    vectors = np.array([p['position'] for p in principles])
    n = len(vectors)
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                v1 = vectors[i] / (np.linalg.norm(vectors[i]) + 1e-6)
                v2 = vectors[j] / (np.linalg.norm(vectors[j]) + 1e-6)
                matrix[i, j] = abs(np.dot(v1, v2))
    score = 1 - np.mean(matrix[matrix > 0]) if matrix.any() else 1.0
    return score

score = compute_ortho(principles_data)
print(f"📊 Orthogonalité: {score:.3f}")
print(f"{'✅ PASS' if score > 0.6 else '⚠️  À REVOIR'}")

📊 Orthogonalité: 0.221
⚠️  À REVOIR


---

## ✅ IDE Complet Fonctionnel

Toutes les fonctionnalités dans un seul notebook !

*TranSysTor v0.2.0*